In [108]:
import torch
from collections import namedtuple

In [109]:
weights_for_each_edge.shape

torch.Size([500])

In [110]:
torch.matmul(weights_for_each_edge,edge_batch_nghbs.unsqueeze(-1)).squeeze()

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.5980e+33, 0.0000e+00])

In [111]:
edge_batch_nghbs.shape

torch.Size([6, 500])

In [112]:
Point = namedtuple("Point", ["x" ,"y"])
issubclass(Point, tuple)

True

In [113]:
pk.shape, edges_view.shape

NameError: name 'pk' is not defined

In [ ]:
edges_view            = edge_batch_nghbs.view(-1, 1, 1, 1)

message_weights      = torch.Tensor(100,500,1).cuda()
pk = (edges_view * message_weights.unsqueeze(0))

In [ ]:
edge_batch_nghbs = hidden_nodes[edge_batch_batch_idc, edge_batch_nghb_idc, :]

NameError: name 'hidden_nodes' is not defined

In [ ]:
edges_view            = edge_batch_nodes.view(-1, 1, 1, 1).cpu()

message_weights = message_weights.cpu()
print(message_weights.shape," ",edges_view.shape)
weights_for_each_edge = (edges_view * message_weights.unsqueeze(0)).sum(3)
print("weights for each edge ",weights_for_each_edge.shape,   " ",node_neighbours.unsqueeze(-1).shape)
node_neighbours = node_neighbours.reshape((100,5,1))
node_neighbours = edge_batch_nghbs
print(node_neighbours.shape,' ',weights_for_each_edge.shape)
torch.matmul(weights_for_each_edge, node_neighbours.unsqueeze(-1)).squeeze()


NameError: name 'edge_batch_nodes' is not defined

In [ ]:
message_weights = 

In [ ]:
message_size = 500
hidden_node_features = 100

class MNN(SummationMpnn):
    """
    The "message neural network" model.
    """
    def __init__(self):
        super().__init__()

        #self.constants       = constants
        message_weights      = torch.Tensor(message_size,
                                            hidden_node_features,
                                            4)
        if True:#== "cuda":
            message_weights = message_weights.to("cuda", non_blocking=True)

        self.message_weights = torch.nn.Parameter(message_weights)

        self.gru             = torch.nn.GRUCell(
            input_size=message_size,
            hidden_size = hidden_node_features,
            bias=True
        )

        self.reset_parameters()

    def reset_parameters(self) -> None:
        stdev = 1.0 / math.sqrt(self.message_weights.size(1))
        self.message_weights.data.uniform_(-stdev, stdev)

    def message_terms(self, nodes,node_neighbours,edges):

        edges_view            = edges.view(-1, 1, 1, 4)
        weights_for_each_edge = (edges_view * self.message_weights.unsqueeze(0)).sum(3)
        print("weights for each edge ",weights_for_each_edge.shape,   " ",node_neighbours.unsqueeze(-1).shape)
        return torch.matmul(weights_for_each_edge,
                            node_neighbours.unsqueeze(-1)).squeeze()

    def update(self, nodes, messages):
        return self.gru(messages, nodes)

class SummationMpnn(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_node_features = 500 #size from constant file
        self.edge_features = 4
        self.message_size = 100
        self.message_passes = 3
    def forward(self,nodes:tuple,edges:torch.tensor)->None:
        adjacency = torch.sum(edges, dim=3)
        print("adj is ",adjacency.shape)
        (edge_batch_batch_idc,
        edge_batch_node_idc,
        edge_batch_nghb_idc) = adjacency.nonzero(as_tuple=True)
        # print("edges various ares",(edge_batch_batch_idc.shape,
        # edge_batch_node_idc.shape,
        # edge_batch_nghb_idc.shape))
        #means picking out non zero waala indiced..gives connecting nodes
        (node_batch_batch_idc,
        node_batch_node_idc) = adjacency.sum(-1).nonzero(as_tuple=True)
        print("nodes various ares",(node_batch_batch_idc,node_batch_node_idc))

        same_batch = node_batch_batch_idc.view(-1,1)== edge_batch_batch_idc
        same_node  = node_batch_node_idc.view(-1, 1) == edge_batch_node_idc
        print("same batch,node",same_batch,   same_node)
        message_summation_matrix = (same_batch * same_node).float()
        print("message ",message_summation_matrix)
        edge_batch_edges = edges[edge_batch_batch_idc, edge_batch_node_idc, edge_batch_nghb_idc, :]
        print("edge_batch_edges ",edge_batch_edges.shape,'hello ',edges.shape)

        hidden_nodes = torch.zeros(nodes.shape[0],
                                   nodes.shape[1],
                                   self.hidden_node_features,
                                   device='cuda')
        hidden_nodes[:nodes.shape[0], :nodes.shape[1], :nodes.shape[2]] = nodes.clone()#padding upto 13 nodes,with features to 100....we have only 9 features here
        node_batch_nodes = hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :]#picking out the same batch

        print("yoyo ",node_batch_nodes.shape)
        print("yoyo2",hidden_nodes.shape)

        for _ in range(self.message_passes):
            edge_batch_nodes = hidden_nodes[edge_batch_batch_idc, edge_batch_node_idc, :]#getting hi  13*13*100

            edge_batch_nghbs = hidden_nodes[edge_batch_batch_idc, edge_batch_nghb_idc, :]#getting neighs(hj)  13*13*1#why its 1
            print("in message pass ",edge_batch_nghbs.shape, "nodes ",edge_batch_nodes.shape)

            message_terms    = self.message_terms(edge_batch_nodes,  
                                                  edge_batch_nghbs,
                                                  edge_batch_edges)

            if len(message_terms.size()) == 1:  # if a single graph in batch
                message_terms = message_terms.unsqueeze(0)

            # the summation in eq. 1 of the NMPQC paper happens here
            messages = torch.matmul(message_summation_matrix, message_terms)

            node_batch_nodes = self.update(node_batch_nodes, messages)
            hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :] = node_batch_nodes.clone()

        node_mask = adjacency.sum(-1) != 0
        return hidden_nodes
        #output    = self.readout(hidden_nodes, nodes, node_mask)

        
        #return output


In [ ]:
(same_batch * same_node).float(),edge_batch_nghb_idc

(tensor([[1., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.]]),
 tensor([1, 0, 2, 1, 2]))

In [ ]:
edge_batch_node_idc,node_batch_node_idc , node_batch_batch_idc.view(-1,1)==edge_batch_batch_idc,"" ,node_batch_node_idc.view(-1,1)==edge_batch_node_idc

(tensor([0, 1, 1, 2, 3]),
 tensor([0, 1, 2, 3]),
 tensor([[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]),
 '',
 tensor([[ True, False, False, False, False],
         [False,  True,  True, False, False],
         [False, False, False,  True, False],
         [False, False, False, False,  True]]))

In [ ]:
edges = torch.tensor(([[[0,1,0,0],
                        [1,0,1,0],
                        [0,1,0,1],
                        [0,0,1,0]]]))
nodes = torch.tensor(([[0,0,0,1],
                        [0,0,1,0],
                        [0,1,0,0],
                        [1,0,0,0]]))
edges.shape,nodes.shape

(torch.Size([1, 4, 4]), torch.Size([4, 4]))

In [118]:
edge_batch_nodes.shape,  edge_batch_nghbs.shape, edge_batch_edges.shape



(torch.Size([6, 500]), torch.Size([6, 500]), torch.Size([6, 1]))

In [122]:
edges_v               = edges.view(-1, 4, 1)
node_neighbours_v     = edges_v * edge_batch_nghbs.view(-1,
                                                        1,500)
node_neighbours_v
#get 4 different kinds of mlp and then pass them one by one with node neighs neighs eembedding and the output will be something like of a 4,[]
# then mulitply edges_v[:i,] 
# terms_masked_per_edge = [
#     edges_v[:, i, :] * self.msg_nns[i](node_neighbours_v[:, i, :])
#     for i in range(self.constants.n_edge_features)
# ]

RuntimeError: The size of tensor a (4) must match the size of tensor b (6) at non-singleton dimension 0

In [ ]:
adjacency = torch.sum(edges, dim=3)
print("adj is ",adjacency.shape)
(edge_batch_batch_idc,
edge_batch_node_idc,
edge_batch_nghb_idc) = adjacency.nonzero(as_tuple=True)
(node_batch_batch_idc,node_batch_node_idc) = adjacency.sum(-1).nonzero(as_tuple=True)


adj is  torch.Size([1, 4, 4])


In [ ]:
edge_batch_node_idc

tensor([0, 1, 1, 2, 3])

In [ ]:
node_batch_node_idc

tensor([0, 1, 2, 3])

In [ ]:
node_batch_node_idc.view(-1,1)," ",edge_batch_node_idc


(tensor([[0],
         [1],
         [2],
         [3]]),
 ' ',
 tensor([0, 1, 1, 2, 3]))

In [ ]:
same_batch = node_batch_batch_idc.view(-1,1)== edge_batch_batch_idc
same_node  = node_batch_node_idc.view(-1, 1) == edge_batch_node_idc
same_node

tensor([[ True, False, False, False, False, False],
        [False,  True,  True, False, False, False],
        [False, False, False,  True,  True, False],
        [False, False, False, False, False,  True]])

In [ ]:
same_batch,edge_batch_batch_idc,edge_batch_node_idc,edge_batch_nghb_idc

(tensor([[True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True]]),
 tensor([0, 0, 0, 0, 0, 0]),
 tensor([0, 1, 1, 2, 2, 3]),
 tensor([1, 0, 2, 1, 3, 2]))

In [ ]:
edges = edges.reshape((1,4,4,1))
nodes = nodes.reshape(1,4,4)

In [ ]:
edge_batch_edges = edges[edge_batch_batch_idc, edge_batch_node_idc, edge_batch_nghb_idc, :]
edge_batch_edges
# hidden_nodes = torch.zeros(nodes.shape[0],
#                                    nodes.shape[1],
#                                    100,
#                                    device='cuda')
# hidden_nodes[:nodes.shape[0], :nodes.shape[1], :nodes.shape[2]] = nodes.clone()

tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1]])

In [ ]:
hidden_nodes

tensor([[[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,


In [ ]:
h1 = hidden_nodes[edge_batch_batch_idc, edge_batch_node_idc, :]

hidden_nodes[0][3].all()==h1[0].all()
print(hidden_nodes.shape," ",h1.shape)

torch.Size([1, 4, 100])   torch.Size([5, 100])


In [ ]:
#edge_batch_node_idc
edge_batch_nghbs = hidden_nodes[edge_batch_batch_idc, edge_batch_nghb_idc, :]
edge_batch_nghbs.shape


torch.Size([5, 100])

In [ ]:
edge_batch_node_idc

tensor([0, 1, 1, 2, 3])

In [ ]:
edges_view = edge_batch_nodes.view(-1, 1, 1, 1)
edges_view.shape
message_weights  = torch.ones((100,500,1)).cuda()
# edges_view*message_weights.unsqueeze(0)

NameError: name 'edge_batch_nodes' is not defined

In [ ]:
pk = edges_view*message_weights.unsqueeze(0)
pk.shape

torch.Size([500, 100, 500, 1])

In [ ]:
pm = torch.tensor([1,2,3])#1,3
als = torch.ones((1,3,1))
(pm*als).sum(2),(pm*als).shape,als,pm

(tensor([[6., 6., 6.]]),
 torch.Size([1, 3, 3]),
 tensor([[[1.],
          [1.],
          [1.]]]),
 tensor([1, 2, 3]))

In [ ]:
edge_batch_nodes.shape

torch.Size([5, 100])

In [ ]:
edge_batch_nodes = hidden_nodes[edge_batch_batch_idc, edge_batch_node_idc, :]


NameError: name 'hidden_nodes' is not defined

In [ ]:
edge_batch_nghb_idc

tensor([1, 0, 2, 1, 2])

In [ ]:
edges_view  = edge_batch_nodes.view(-1, 1, 1,1)


In [ ]:
message_terms(edge_batch_nodes,edge_batch_nghbs,edge_batch_edges)

In [ ]:
message_summation_matrix = (same_batch * same_node).float()
message_summation_matrix

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])

In [ ]:
edge_batch_batch_idc,edge_batch_node_idc,edge_batch_nghb_idc," node node ",node_batch_batch_idc,node_batch_node_idc


(tensor([0, 0, 0, 0, 0]),
 tensor([0, 1, 1, 2, 3]),
 tensor([1, 0, 2, 1, 2]),
 ' node node ',
 tensor([0, 0, 0, 0]),
 tensor([0, 1, 2, 3]))

In [ ]:
adjacency

tensor([[[0, 1, 0, 0],
         [1, 0, 1, 0],
         [0, 1, 0, 0],
         [0, 0, 1, 0]]])

In [ ]:
hidden_nodes.shape

torch.Size([1, 4, 4, 4])

In [ ]:
mnn = MNN()
node = nodes.cuda()
edges=torch.zeros((1,13,13,4)).cuda()
#edges= torch.tensor([[[[1, 1, 0, 0],[1, 1, 0, 0],[1, 0, 1, 0],[1, 0, 0,1]]]])
#1node with 4 edges,but no info on other edges
edges = edges.cuda()
mnn(nodes,edges)

adj is  torch.Size([1, 13, 13])
nodes various ares (tensor([], device='cuda:0', dtype=torch.int64), tensor([], device='cuda:0', dtype=torch.int64))
same batch,node tensor([], device='cuda:0', size=(0, 0), dtype=torch.bool) tensor([], device='cuda:0', size=(0, 0), dtype=torch.bool)
message  tensor([], device='cuda:0', size=(0, 0))
edge_batch_edges  torch.Size([0, 4]) hello  torch.Size([1, 13, 13, 4])
yoyo  torch.Size([0, 500])
yoyo2 torch.Size([1, 13, 500])
in message pass  torch.Size([0, 500]) nodes  torch.Size([0, 500])
weights for each edge  torch.Size([0, 500, 100])   torch.Size([0, 500, 1])


RuntimeError: batch1 dim 2 must match batch2 dim 1

In [ ]:
pk[0][5]=1

In [ ]:
nn[0],pk[0]

(tensor([0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
pk[0].shape
nn[0].view(-1,1)==pk[0]
ash = torch.tensor([[[1, 1, 1, 0, 1],[1, 1, 1, 0, 1],[1, 1, 1, 0, 1],[1, 1, 1, 0, 1]]])
nn = ash.sum(-1)
pk = ash.nonzero(as_tuple=True)
nn = nn.nonzero(as_tuple=True)
nn[0].view(-1,1).shape



tensor([[ True,  True,  True,  True,  True, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True]])

adj is  torch.Size([1, 13, 13])
edges various ares (torch.Size([169]), torch.Size([169]), torch.Size([169]))
nodes various ares (tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], device='cuda:0'))
yoyo  torch.Size([13, 500])
yoyo2 torch.Size([1, 13, 500])
in message pass  torch.Size([169, 500]) nodes  torch.Size([169, 500])
weights for each edge  torch.Size([169, 500, 100])   torch.Size([169, 500, 1])


RuntimeError: batch1 dim 2 must match batch2 dim 1

In [ ]:
from torch_geometric.nn import MessagePassing
import torch
from collections import namedtuple

In [ ]:
mnn = MNN()
node = nodes.cuda()
edges = edges.cuda()
mnn(nodes,edges)

yoyo  torch.Size([13, 500])
yoyo2 torch.Size([1, 13, 500])
weights for each edge  torch.Size([169, 500, 100])   torch.Size([169, 500, 1])


RuntimeError: batch1 dim 2 must match batch2 dim 1

In [ ]:
nodes = torch.zeros((1,13,9))
edges = torch.ones((1,13,13,4))
#edges = edges.type(torch.LongTensor)


In [ ]:
mpnn(nodes,edges)

TypeError: Required parameter nodes is empty.

In [ ]:
mpnn

MPNN(
  (lin): Linear(in_features=9, out_features=20, bias=True)
  (GRU): GRUCell(500, 100)
)

In [ ]:
import torch.nn as nn
import math

In [ ]:
class MNN(SummationMpnn):
    def __init__(self,constant):
        super().__init__(constant)
        self.constant = constant
        message_weights = torch.Tensor(self.constants.message_size,
                                            self.constants.hidden_node_features,
                                            self.constants.n_edge_features)
        self.message_weights = nn.Parameter(message_weights)
        self.gru = torch.nn.GRUCell(
            input_size=self.constant.message_size,
            hidden_size=500,bias = True
        )
        self.ADPReadout = Readout()
        
        def reset_parameters(self) -> None:
            stdev = 1.0 / math.sqrt(self.message_weights.size(1))
            self.message_weights.data.uniform_(-stdev, stdev)

        def update(self, nodes : torch.Tensor, messages : torch.Tensor) -> torch.Tensor:
            return self.gru(messages, nodes)    

        def readout(self, hidden_nodes : torch.Tensor, input_nodes : torch.Tensor,
                node_mask : torch.Tensor) -> torch.Tensor:
            graph_embeddings = torch.sum(hidden_nodes, dim=1)
            output           = self.APDReadout(hidden_nodes, graph_embeddings)
            return output

            




NameError: name 'SummationMpnn' is not defined

In [ ]:
adjacency = torch.sum(edges, dim=3)

# **note: "idc" == "indices", "nghb{s}" == "neighbour(s)"
(edge_batch_batch_idc,
    edge_batch_node_idc,
    edge_batch_nghb_idc) = adjacency.nonzero(as_tuple=True)

(node_batch_batch_idc, node_batch_node_idc) = adjacency.sum(-1).nonzero(as_tuple=True)

same_batch = node_batch_batch_idc.view(-1, 1) == edge_batch_batch_idc
same_node  = node_batch_node_idc.view(-1, 1) == edge_batch_node_idc

# element ij of `message_summation_matrix` is 1 if `edge_batch_edges[j]`
# is connected with `node_batch_nodes[i]`, else 0
message_summation_matrix = (same_batch * same_node).float()

edge_batch_edges = edges[edge_batch_batch_idc, edge_batch_node_idc, edge_batch_nghb_idc, :]

# pad up the hidden nodes
hidden_nodes = torch.zeros(nodes.shape[0],
                            nodes.shape[1],
                            500)
hidden_nodes[:nodes.shape[0], :nodes.shape[1], :nodes.shape[2]] = nodes.clone()
# message_weights = 
node_batch_nodes = hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :]
message_passes = 4
for _ in range(message_passes):
    edge_batch_nodes = hidden_nodes[edge_batch_batch_idc, edge_batch_node_idc, :]

    edge_batch_nghbs = hidden_nodes[edge_batch_batch_idc, edge_batch_nghb_idc, :]

    edges_view            = edge_batch_nodes.view(-1, 1, 1, 4)
    # weights_for_each_edge = (edges_view * message_weights.unsqueeze(0)).sum(3)
    # print("weights for each edge ",weights_for_each_edge.shape,   " ",node_neighbours.unsqueeze(-1).shape)
    # return torch.matmul(weights_for_each_edge,
    #                     edge_batch_nghbs.unsqueeze(-1)).squeeze()

    # message_terms    = message_terms(edge_batch_nodes,
                                            # edge_batch_nghbs,
                                            # edge_batch_edges)


In [ ]:
message_summation_matrix.shape,#size of message should be 6*500-->at last so that we can get--->[4,6]*[6,1,500] --->[4,500]

message_summation_matrix

tensor([[1., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1.]])

In [ ]:
nodes = nodes.reshape(4,4,1)
message_weights = torch.Tensor(500)#Lets multiply each of from 500 with some number
edges_view = edge_batch_edges.view(-1, 1, 1, 1)
# weights_for_each_edge = 
weights_for_each_edge = message_weights#(edges_view * message_weights.unsqueeze(0)).sum(3) #weight for a signle edge 
print("weights for each edge ",weights_for_each_edge.shape,   " ",edge_batch_nghbs.shape)
#so what we are doing is--->we are multiplying each neigh depecited whole as matrix(500,6,1) with  
ashish = (edge_batch_nghbs*weights_for_each_edge)#.squeeze()
print(ashish.shape)

weights for each edge  torch.Size([500])   torch.Size([6, 500])
torch.Size([6, 500])


In [ ]:
message = torch.matmul(message_summation_matrix,ashish)
print(message.shape)

torch.Size([4, 500])


In [ ]:
gru  = torch.nn.GRUCell(
            input_size=500,#message size
            hidden_size=500,#hidden node features,
            bias=True)  #nodes are having size->[4,500],edges->

In [ ]:
hyperparameters = {
        "mlp1_depth"          : 4,
        "mlp1_dropout_p"      : 0.0,
        "mlp1_hidden_dim"     : 500,
        "mlp2_depth"          : 4,
        "mlp2_dropout_p"      : 0.0,
        "mlp2_hidden_dim"     : 500,
        "hidden_node_features": 100,
        "message_passes"      : 3,
        "message_size"        : 100,
    }

In [ ]:
gru(node_batch_nodes,message).shape

torch.Size([4, 500])

In [ ]:
edge_batch_batch_idc,edge_batch_node_idc,edge_batch_nghb_idc,message_weights.shape," ",edges_view.shape

(tensor([0, 0, 0, 0, 0, 0]),
 tensor([0, 1, 1, 2, 2, 3]),
 tensor([1, 0, 2, 1, 3, 2]),
 torch.Size([100, 500, 1]),
 ' ',
 torch.Size([6, 1, 1, 1]))

In [ ]:
edge_batch_edges = edges[edge_batch_batch_idc, edge_batch_node_idc, edge_batch_nghb_idc, :]


In [ ]:
edge_batch_edges.view(-1,1,1,1).shape

torch.Size([6, 1, 1, 1])

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, in_features,hidden_layer_sizes, out_features,
                 dropout_p):
        super().__init__()

        activation_function = torch.nn.SELU
        # create list of all layer feature sizes
        fs = [in_features, *hidden_layer_sizes, out_features]
        # create list of linear_blocks
        layers = [self._linear_block(in_f, out_f,
                                     activation_function,
                                     dropout_p)
                  for in_f, out_f in zip(fs, fs[1:])]
        # concatenate modules in all    sequentials in layers list
        layers = [module for sq in layers for module in sq.children()]

        # add modules to sequential container
        self.seq = torch.nn.Sequential(*layers)

    def _linear_block(self, in_f : int, out_f : int, activation : torch.nn.Module,
                      dropout_p : float) -> torch.nn.Sequential:
        
        # bias must be used in most MLPs in our models to learn from empty graphs
        linear = torch.nn.Linear(in_f, out_f, bias=True)
        torch.nn.init.xavier_uniform_(linear.weight)
        return torch.nn.Sequential(linear, activation(), torch.nn.AlphaDropout(dropout_p))

    def forward(self, layers_input : torch.nn.Sequential) -> torch.nn.Sequential:
        return self.seq(layers_input)
hyperparameters = {
        "enn_depth"            : 4,
        "enn_dropout_p"        : 0.0,
        "enn_hidden_dim"       : 250,
        "mlp1_depth"           : 4,
        "mlp1_dropout_p"       : 0.0,
        "mlp1_hidden_dim"      : 500,
        "mlp2_depth"           : 4,
        "mlp2_dropout_p"       : 0.0,
        "mlp2_hidden_dim"      : 500,
        "hidden_node_features" : 100,
        "message_passes"       : 3,
        "message_size"         : 100,
        "s2v_lstm_computations": 3,
        "s2v_memory_size"      : 100,
    }

In [ ]:
class GlobalReadout(torch.nn.Module):
    def __init__(self, f_add_elems : int, f_conn_elems : int, f_term_elems : int,
                 mlp1_depth : int, mlp1_dropout_p : float, mlp1_hidden_dim : int,
                 mlp2_depth : int, mlp2_dropout_p : float, mlp2_hidden_dim : int,
                 graph_emb_size : int, max_n_nodes : int, node_emb_size : int,
                 device : str) -> None:
        super().__init__()

        self.device = device

        # preliminary f_add
        self.fAddNet1 = MLP(
            in_features=node_emb_size,
            hidden_layer_sizes=[mlp1_hidden_dim] * mlp1_depth,
            out_features=f_add_elems,
            dropout_p=mlp1_dropout_p
        )

        # preliminary f_conn
        self.fConnNet1 = MLP(
            in_features=node_emb_size,
            hidden_layer_sizes=[mlp1_hidden_dim] * mlp1_depth,
            out_features=f_conn_elems,
            dropout_p=mlp1_dropout_p
        )

        # final f_add
        self.fAddNet2 = MLP(
            in_features=(max_n_nodes * f_add_elems + graph_emb_size),
            hidden_layer_sizes=[mlp2_hidden_dim] * mlp2_depth,
            out_features=f_add_elems * max_n_nodes,
            dropout_p=mlp2_dropout_p
        )

        # final f_conn
        self.fConnNet2 = MLP(
            in_features=(max_n_nodes * f_conn_elems + graph_emb_size),
            hidden_layer_sizes=[mlp2_hidden_dim] * mlp2_depth,
            out_features=f_conn_elems * max_n_nodes,
            dropout_p=mlp2_dropout_p
        )

        # final f_term (only takes as input graph embeddings)
        self.fTermNet2 = MLP(
            in_features=graph_emb_size,
            hidden_layer_sizes=[mlp2_hidden_dim] * mlp2_depth,
            out_features=f_term_elems,
            dropout_p=mlp2_dropout_p
        )

    def forward(self, node_level_output : torch.Tensor,
                graph_embedding_batch : torch.Tensor) -> torch.Tensor:
        """
        Defines forward pass.
        """
        if self.device == "cuda":
            self.fAddNet1  = self.fAddNet1.to("cuda", non_blocking=True)
            self.fConnNet1 = self.fConnNet1.to("cuda", non_blocking=True)
            self.fAddNet2  = self.fAddNet2.to("cuda", non_blocking=True)
            self.fConnNet2 = self.fConnNet2.to("cuda", non_blocking=True)
            self.fTermNet2 = self.fTermNet2.to("cuda", non_blocking=True)

        # get preliminary f_add and f_conn
        f_add_1  = self.fAddNet1(node_level_output)
        f_conn_1 = self.fConnNet1(node_level_output)

        if self.device == "cuda":
            f_add_1  = f_add_1.to("cuda", non_blocking=True)
            f_conn_1 = f_conn_1.to("cuda", non_blocking=True)

        # reshape preliminary APDs into flattenened vectors (e.g. one vector per
        # graph in batch)
        f_add_1_size  = f_add_1.size()
        f_conn_1_size = f_conn_1.size()
        f_add_1  = f_add_1.view((f_add_1_size[0], f_add_1_size[1] * f_add_1_size[2]))
        f_conn_1 = f_conn_1.view((f_conn_1_size[0], f_conn_1_size[1] * f_conn_1_size[2]))

        # get final f_add, f_conn, and f_term
        f_add_2 = self.fAddNet2(
            torch.cat((f_add_1, graph_embedding_batch), dim=1).unsqueeze(dim=1)
        )
        f_conn_2 = self.fConnNet2(
            torch.cat((f_conn_1, graph_embedding_batch), dim=1).unsqueeze(dim=1)
        )
        f_term_2 = self.fTermNet2(graph_embedding_batch)

        if self.device == "cuda":
            f_add_2  = f_add_2.to("cuda", non_blocking=True)
            f_conn_2 = f_conn_2.to("cuda", non_blocking=True)
            f_term_2 = f_term_2.to("cuda", non_blocking=True)

        # flatten and concatenate
        cat = torch.cat((f_add_2.squeeze(dim=1), f_conn_2.squeeze(dim=1), f_term_2), dim=1)

        return cat  # note: no activation function before returning


In [ ]:
fConnNet2

MLP(
  (seq): Sequential(
    (0): Linear(in_features=165, out_features=500, bias=True)
    (1): SELU()
    (2): AlphaDropout(p=0.0, inplace=False)
    (3): Linear(in_features=500, out_features=500, bias=True)
    (4): SELU()
    (5): AlphaDropout(p=0.0, inplace=False)
    (6): Linear(in_features=500, out_features=500, bias=True)
    (7): SELU()
    (8): AlphaDropout(p=0.0, inplace=False)
    (9): Linear(in_features=500, out_features=500, bias=True)
    (10): SELU()
    (11): AlphaDropout(p=0.0, inplace=False)
    (12): Linear(in_features=500, out_features=65, bias=True)
    (13): SELU()
    (14): AlphaDropout(p=0.0, inplace=False)
  )
)

In [ ]:
fAddNet1

MLP(
  (seq): Sequential(
    (0): Linear(in_features=500, out_features=500, bias=True)
    (1): SELU()
    (2): AlphaDropout(p=0.0, inplace=False)
    (3): Linear(in_features=500, out_features=500, bias=True)
    (4): SELU()
    (5): AlphaDropout(p=0.0, inplace=False)
    (6): Linear(in_features=500, out_features=500, bias=True)
    (7): SELU()
    (8): AlphaDropout(p=0.0, inplace=False)
    (9): Linear(in_features=500, out_features=500, bias=True)
    (10): SELU()
    (11): AlphaDropout(p=0.0, inplace=False)
    (12): Linear(in_features=500, out_features=250, bias=True)
    (13): SELU()
    (14): AlphaDropout(p=0.0, inplace=False)
  )
)